In [3]:
from tensorflow.keras.datasets import mnist
from sklearn.linear_model import LogisticRegression
import joblib

print("Loading MNIST from Keras...")
(X_train, y_train), _ = mnist.load_data()

# Flatten 28x28 -> 784
X_train = X_train.reshape((X_train.shape[0], -1)).astype("float32") / 255.0

print("Training Logistic Regression...")
model = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
model.fit(X_train, y_train)

print("Saving model...")
joblib.dump(model, "mnist_logreg.pkl")
print("Model saved as mnist_logreg.pkl")

Loading MNIST from Keras...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training Logistic Regression...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Saving model...
Model saved as mnist_logreg.pkl


In [5]:
import numpy as np
import gzip
from sklearn.linear_model import LogisticRegression
import joblib

def load_images(filename):
    with gzip.open(filename, 'rb') as f:
        f.read(16)  # skip magic number & dimensions
        data = np.frombuffer(f.read(), dtype=np.uint8)
        data = data.reshape(-1, 28*28) / 255.0
    return data

def load_labels(filename):
    with gzip.open(filename, 'rb') as f:
        f.read(8)  # skip magic number & items
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

print("Loading MNIST from local .gz files...")
X_train = load_images("train-images-idx3-ubyte.gz")
y_train = load_labels("train-labels-idx1-ubyte.gz")

print("Training Logistic Regression...")
model = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    multi_class='multinomial',
    verbose=1
)
model.fit(X_train, y_train)

print("Saving model...")
joblib.dump(model, "mnist_2.pkl")
print("✅ Model saved as mnist_2.pkl")

Loading MNIST from local .gz files...
Training Logistic Regression...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Saving model...
✅ Model saved as mnist_2.pkl


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished


In [6]:
from sklearn.datasets import load_digits
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Load digits dataset (8x8 images)
digits = load_digits()
X = digits.data / 16.0  # Normalize to 0-1
y = digits.target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a small neural net
mlp = MLPClassifier(hidden_layer_sizes=(64,), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Accuracy
y_pred = mlp.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, y_pred))

# Save the model
joblib.dump(mlp, "digits_mlp.pkl")
print("Model saved as digits_mlp.pkl")

Test accuracy: 0.9722222222222222
Model saved as digits_mlp.pkl


In [10]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib
import gzip
import os

# ---------- Function to load MNIST from local .gz files ----------
def load_mnist_local(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, f'{kind}-labels-idx1-ubyte.gz')
    images_path = os.path.join(path, f'{kind}-images-idx3-ubyte.gz')

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 28*28)

    return images, labels

# ---------- Load data ----------
# You can download from: http://yann.lecun.com/exdb/mnist/
# Place the 4 .gz files in a folder 'mnist_data'
X_train, y_train = load_mnist_local('./', kind='train')
X_test, y_test = load_mnist_local('./', kind='t10k')

# Normalize
X_train = X_train / 255.0
X_test = X_test / 255.0

# ---------- Train MLP ----------
mlp = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),  # deeper network
    activation='relu',
    solver='adam',
    alpha=1e-4,  # L2 regularization
    batch_size=128,
    learning_rate_init=0.001,
    max_iter=50,
    early_stopping=True,
    n_iter_no_change=5,
    random_state=42
)

mlp.fit(X_train, y_train)

# ---------- Evaluate ----------
y_pred = mlp.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, y_pred))

# ---------- Save model ----------
joblib.dump(mlp, "mnist_mlp2.pkl")
print("Model saved as mnist_mlp.pkl")


Test accuracy: 0.9789
Model saved as mnist_mlp.pkl


In [9]:
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

print("Loading MNIST...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

# Normalize pixel values
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Better MLP
mlp = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),  # deeper network
    activation='relu',
    solver='adam',
    alpha=1e-4,  # L2 regularization
    batch_size=128,
    learning_rate_init=0.001,
    max_iter=50,
    early_stopping=True,
    n_iter_no_change=5,
    random_state=42
)

print("Training...")
mlp.fit(X_train, y_train)

# Evaluate
train_acc = mlp.score(X_train, y_train)
test_acc = mlp.score(X_test, y_test)
print(f"Train Accuracy: {train_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Save both scaler and model
joblib.dump((scaler, mlp), "mnist_mlp_better.pkl")
print("Model saved as mnist_mlp_better.pkl")


Loading MNIST...


/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:110: UserWarning: A network error occurred while downloading https://api.openml.org/data/v1/download/52667. Retrying...
  warn(


HTTPError: HTTP Error 404: Not Found